In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import json
import glob
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from IPython import display
from tqdm.notebook import tqdm

# Data structures and functions for rendering
from pytorch3d.structures import Volumes
from pytorch3d.transforms import so3_exp_map
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    NDCMultinomialRaysampler,
    MonteCarloRaysampler,
    EmissionAbsorptionRaymarcher,
    ImplicitRenderer,
    RayBundle,
    ray_bundle_to_ray_points,
)
import sys
sys.path.append('/home/azhuavlev/PycharmProjects/ml-neuman_mano/pytorch3d_nerf')
from mano_custom import mano_pytorch3d

device = torch.device("cuda:0")

In [2]:
hand_model = mano_pytorch3d.create_mano_custom(return_right_hand=False)

Fix shapedirs bug of MANO


In [3]:
import trimesh

faces = torch.from_numpy(hand_model.faces.astype(np.int32))[None, :, :]
verts_zero_pose = hand_model.get_flat_hand_vertices_pytorch3d('cpu')

mesh = trimesh.Trimesh(vertices=verts_zero_pose[0], faces=faces[0])
boundary_vertices = [92, 38, 122, 118, 117, 119, 120, 108, 79, 78, 121, 214, 215, 279, 239, 234][::-1]

centroid = mesh.vertices[boundary_vertices].mean(axis=0)
vertices_repaired = np.append(verts_zero_pose[0], centroid[None, :], axis=0)

new_faces = []
for i in range(len(boundary_vertices) - 1):
   next_i = i + 1 % len(boundary_vertices)
   new_faces.append([len(vertices_repaired) - 1, boundary_vertices[i], boundary_vertices[next_i]])
new_faces.append([len(vertices_repaired) - 1, boundary_vertices[len(boundary_vertices)-1], boundary_vertices[0]])

faces_repaired = np.append(faces[0], new_faces, axis=0)

mesh_repaired = trimesh.Trimesh(vertices=vertices_repaired, faces=faces_repaired)
print(mesh_repaired.is_watertight)

True


In [4]:
vertices_repaired = torch.tensor(vertices_repaired)
faces_repaired = torch.tensor(faces_repaired)

In [5]:
pts_x = torch.linspace(-0.15, 0.15, 100)

points = torch.stack(torch.meshgrid(pts_x, pts_x, pts_x), dim=-1).reshape(1, -1, 3)
points.shape

/scratch_net/manifoldnet/azhuavlev/conda_envs/neuman_kaolin/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 1000000, 3])

In [6]:
import kaolin

is_inside = kaolin.ops.mesh.check_sign(vertices_repaired.to('cuda'), faces_repaired.to('cuda'), points.to('cuda'))
is_inside

RuntimeError: CUDA driver initialization failed, you might not have a CUDA gpu.

In [7]:
torch.cuda.is_available()

/scratch_net/manifoldnet/azhuavlev/conda_envs/neuman_kaolin/lib/python3.9/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False